In [ ]:
!pip install midiutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54567 sha256=e7ce278ada6919542464e8e6ddeb1e047d65c88a136151acbd3e11d7cc3f269f
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil


In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from midiutil import MIDIFile
import pickle

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/MyDrive/ChantAI')

Mounted at /content/drive


In [ ]:
def note_to_midi(note, octave):
    # MIDI note numbers for the notes in octave 0
    note_map = {
        'c': 0, 'c#': 1, 'd': 2, 'd#': 3, 'e': 4, 'f': 5, 'f#': 6,
        'g': 7, 'g#': 8, 'a': 9, 'a#': 10, 'b': 11
    }

    # Convert note to lowercase to handle both upper and lower case inputs
    note = note.lower()

    # Calculate the MIDI number
    midi_number = (octave + 1) * 12 + note_map[note]

    return midi_number

def melody_to_midi(melody, rhythm_pattern, velocity_pattern, filename):
    midi = MIDIFile(1)
    midi.addTempo(0, 0, 120)

    for i, note in enumerate(melody):
        pitch_class = note[:-1]
        octave = note[-1]
        pitch = note_to_midi(pitch_class.lower(), int(octave))
        duration = rhythm_pattern[i % len(rhythm_pattern)]
        velocity = velocity_pattern[i % len(velocity_pattern)]
        midi.addNote(0, 0, pitch, i, duration, velocity)  # Add note with duration of 1

    with open(filename, 'wb') as output_file:
        midi.writeFile(output_file)

In [ ]:
tb_dorian = pd.read_csv('data/antiphon_melodies.csv')
tb_dorian = tb_dorian[tb_dorian['mode'].isin([str(x) for x in range(1, 9)])].dropna()
unique_ids = tb_dorian.groupby('id')['notes'].apply(list).reset_index().drop_duplicates(subset = 'notes')['id'].tolist()
tb_dorian = tb_dorian[tb_dorian['id'].isin(unique_ids)]
test_set = np.random.choice(unique_ids, 100, replace = False)
train_data = tb_dorian[~tb_dorian['id'].isin(test_set)]
test_data = tb_dorian[tb_dorian['id'].isin(test_set)]
melodies = train_data.groupby('id')['notes'].apply(list).tolist()
modes = train_data.groupby('id')['mode'].first().to_list()

In [ ]:
pd.crosstab(tb_dorian.loc[tb_dorian['notes_to_end'] == 0, 'notes'], tb_dorian.loc[tb_dorian['notes_to_end'] == 0, 'mode'], normalize = "columns")

mode,1,2,3,4,5,6,7,8
notes,,,,,,,,
A3,0.000743,0.001531,0.001664,0.000574,0.000000,0.000000,0.000366,0.000000
A4,0.200248,0.030628,0.365225,0.119977,0.448276,0.100119,0.255771,0.086403
A5,0.000000,0.000000,0.000000,0.000000,0.000000,0.001192,0.000000,0.000199
B3,0.001239,0.000766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B4,0.005452,0.000766,0.137271,0.016648,0.020990,0.008343,0.063027,0.024288
C4,0.017348,0.043645,0.000000,0.016073,0.002999,0.020262,0.001099,0.001593
C5,0.006444,0.002297,0.139767,0.010333,0.212894,0.017878,0.227556,0.168425
D3,0.000000,0.000766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D4,0.284511,0.745789,0.008319,0.051665,0.004498,0.008343,0.006229,0.009556


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Create a vocabulary and mode dictionary
vocab = sorted(set([note for melody in melodies for note in melody]))
vocab_dict = {note: i + 1 for i, note in enumerate(vocab)}  # Start indexing from 1 for padding
vocab_size = len(vocab_dict) + 1  # +1 for padding

mode_dict = {mode: i + 1 for i, mode in enumerate(sorted(set(modes)))}
mode_vocab_size = len(mode_dict) + 1

# Encode melodies and modes using dictionaries
encoded_melodies = [[vocab_dict[note] for note in melody] for melody in melodies]
encoded_modes = [mode_dict[mode] for mode in modes]

# Create position input indicating how many elements are left until the end of the series
positions = [[len(melody) - idx - 1 for idx in range(len(melody))] for melody in encoded_melodies]

# Prepare data for the model
X_series = []
X_modes = []
X_positions = []
y = []

for melody, mode, position in zip(encoded_melodies, encoded_modes, positions):
    for i in range(1, len(melody)):
        X_series.append(melody[:i])
        X_modes.append([mode])
        X_positions.append([position[i-1]])
        y.append(melody[i])

# Pad sequences to have the same length
X_series = tf.keras.preprocessing.sequence.pad_sequences(X_series, padding='pre')

# Convert to numpy arrays
X_series = np.array(X_series)
X_modes = np.array(X_modes)
X_positions = np.array(X_positions)
y = np.array(y)

# Define cosine decay function for learning rate
def cosine_decay(epoch, initial_lr):
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / epochs))
    return initial_lr * cosine_decay

# Parameters
embedding_dim = 16
lstm_units = 32
mode_embedding_dim = 4
dense_units = 64
dropout_rate = 0.5
initial_lr = 0.001
epochs = 6

# Input layers
series_input = Input(shape=(None,), name='series_input')
mode_input = Input(shape=(1,), name='mode_input')
position_input = Input(shape=(1,), name='position_input')

# Embedding layers
series_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='series_embedding')(series_input)
mode_embedding = Embedding(input_dim=mode_vocab_size, output_dim=mode_embedding_dim, name='mode_embedding')(mode_input)

# LSTM layer
lstm_output = LSTM(units=lstm_units, name='lstm_layer')(series_embedding)

# Concatenate embeddings and position input
concatenated = Concatenate(name='concat_layer')([lstm_output, tf.squeeze(mode_embedding, axis=1), position_input])

# Dense layers with dropout and batch normalization
dense1 = Dense(units=dense_units, activation='relu', name='dense1')(concatenated)
dropout1 = Dropout(rate=dropout_rate, name='dropout1')(dense1)
batch_norm1 = BatchNormalization(name='batch_norm1')(dropout1)

dense2 = Dense(units=dense_units, activation='relu', name='dense2')(batch_norm1)
dropout2 = Dropout(rate=dropout_rate, name='dropout2')(dense2)
batch_norm2 = BatchNormalization(name='batch_norm2')(dropout2)

# Output layer
output = Dense(units=vocab_size, activation='softmax', name='output')(batch_norm2)

# Model
model = Model(inputs=[series_input, mode_input, position_input], outputs=output, name='LSTM_Model')

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(schedule=lambda epoch: cosine_decay(epoch, initial_lr))

# Summary of the model
model.summary()

Model: "LSTM_Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 series_input (InputLayer)   [(None, None)]               0         []                            
                                                                                                  
 mode_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 series_embedding (Embeddin  (None, None, 16)             400       ['series_input[0][0]']        
 g)                                                                                               
                                                                                                  
 mode_embedding (Embedding)  (None, 1, 4)                 36        ['mode_input[0][0]'] 

In [ ]:
# Fit model
model.fit([X_series, X_modes, X_positions], y,
          validation_split=0.2, epochs=epochs, batch_size=64,
          callbacks=[early_stopping, lr_scheduler])


Epoch 1/6
8295/8295 [==============================] - 362s 43ms/step - loss: 1.6036 - accuracy: 0.3681 - val_loss: 1.4083 - val_accuracy: 0.4332 - lr: 0.0010
Epoch 2/6
8295/8295 [==============================] - 368s 44ms/step - loss: 1.4563 - accuracy: 0.4227 - val_loss: 1.3505 - val_accuracy: 0.4661 - lr: 9.3301e-04
Epoch 3/6
8295/8295 [==============================] - 356s 43ms/step - loss: 1.4132 - accuracy: 0.4506 - val_loss: 1.3159 - val_accuracy: 0.4877 - lr: 7.5000e-04
Epoch 4/6
8295/8295 [==============================] - 354s 43ms/step - loss: 1.3851 - accuracy: 0.4670 - val_loss: 1.2929 - val_accuracy: 0.5011 - lr: 5.0000e-04
Epoch 5/6
8295/8295 [==============================] - 368s 44ms/step - loss: 1.3694 - accuracy: 0.4772 - val_loss: 1.2797 - val_accuracy: 0.5059 - lr: 2.5000e-04
Epoch 6/6
8295/8295 [==============================] - 369s 44ms/step - loss: 1.3611 - accuracy: 0.4829 - val_loss: 1.2746 - val_accuracy: 0.5101 - lr: 6.6987e-05


In [ ]:
# Save the model
model.save('models/v2/melody_prediction_model_with_notes_until_end.h5')

# Save the dictionaries
with open('models/v2/pitch_encoder.pkl', 'wb') as f:
    pickle.dump(vocab_dict, f)

with open('models/v2/mode_encoder.pkl', 'wb') as f:
    pickle.dump(mode_dict, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Function to load the model and dictionaries
def load_resources(model_path, vocab_dict_path, mode_dict_path):
    model = load_model(model_path)
    with open(vocab_dict_path, 'rb') as f:
        vocab_dict = pickle.load(f)
    with open(mode_dict_path, 'rb') as f:
        mode_dict = pickle.load(f)
    return model, vocab_dict, mode_dict

# Function to predict the element probabilities for a given sequence, mode, and remaining notes
def predict_element_probabilities(sequence, mode, notes_left, model, vocab_dict, mode_dict):
    # Reverse the vocab_dict to get the reverse mapping from indices to elements
    index_to_vocab = {index: element for element, index in vocab_dict.items()}

    # Encode the sequence and mode using the dictionaries
    encoded_sequence = [vocab_dict[element] for element in sequence]
    encoded_mode = np.array([mode_dict[mode]]).reshape(1, 1)

    # Create the position input based on the notes left in the series
    position_input = np.array([[notes_left]])

    # Pad the sequence to match the input shape of the model
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences([encoded_sequence], padding='pre')

    # Predict the probabilities
    probabilities = model.predict([padded_sequence, encoded_mode, position_input])[0]

    # Map the probabilities back to the elements
    element_probabilities = {index_to_vocab[i]: prob for i, prob in enumerate(probabilities) if i in index_to_vocab}

    return element_probabilities

# Function to extend a sequence to a specified length using temperature-controlled sampling
def extend_sequence(initial_sequence, mode, target_length, temperature, model, vocab_dict, mode_dict):
    sequence = initial_sequence[:]
    while len(sequence) < target_length:
        notes_left = target_length - len(sequence)
        element_probabilities = predict_element_probabilities(sequence, mode, notes_left, model, vocab_dict, mode_dict)

        # Sort elements by probability
        sorted_elements = sorted(element_probabilities.items(), key=lambda item: item[1], reverse=True)

        # Determine the number of elements to sample from based on temperature
        num_elements = int(len(sorted_elements) * temperature)
        num_elements = max(1, num_elements)  # Ensure at least one element is sampled

        # Sample the next element based on the adjusted probabilities
        elements, probabilities = zip(*sorted_elements[:num_elements])
        probabilities = np.array(probabilities) / np.sum(probabilities)  # Normalize probabilities
        next_element = np.random.choice(elements, p=probabilities)

        # Append the next element to the sequence
        sequence.append(next_element)

    return sequence

# Load resources
model, vocab_dict, mode_dict = load_resources('models/v2/melody_prediction_model_with_notes_until_end.h5',
                                              'models/v2/pitch_encoder.pkl',
                                              'models/v2/mode_encoder.pkl')

# Example usage
initial_sequence = ["F4", "E4"]
mode = "1"
target_length = 16
temperature = 0.5  # Specify the temperature value
extended_sequence = extend_sequence(initial_sequence, mode, target_length, temperature, model, vocab_dict, mode_dict)

print("Extended Sequence:", extended_sequence)


1/1 [==============================] - 0s 25ms/step
Extended Sequence: ['F4', 'E4', 'F4', 'E4', 'D4', 'E4', 'D4', 'D4', 'D4', 'C4', 'C4', 'C4', 'E4', 'F4', 'G4', 'A4']


In [ ]:
melody_to_midi(extended_sequence,[1, 1, 1], [127, 70, 100, 50], "generated_melody.mid")